Following this tutorial: https://medium.com/geekculture/how-to-collect-data-from-imdb-explore-it-b669f56b7dfb

First make sure you have the BeautifulSoup package and the Requests package installed
Run this in your command line: 
> pip install beautifulsoup4

> pip install requests

In [43]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import requests
import re

Site to be Scrapped: https://www.imdb.com/search/title/?title_type=tv_series&release_date=1989-01-01,2022-12-31&genres=animation

In [44]:
#url with link to IMDB TV Series, Animation, released between 1989 and 2022
#url = 'https://www.imdb.com/search/title/?title_type=tv_series&release_date=1989-01-01,2022-12-31&genres=animation'
url = "https://www.imdb.com/chart/toptv"
response = get(url)
tv_soup = BeautifulSoup(response.text, 'html.parser')

Not super sure what is happening in the cells below, so I'm just gonna follow the tutorial and see if we need to come back to this later

In [45]:
#collecting info about all the tv shows from the website
containers = tv_soup.find_all('td', class_='titleColumn')

#collecting info about the tv shows' ratings
containers_rating = tv_soup.find_all('td', class_="ratingColumn imdbRating")    

In [46]:
#listing out the ratings of the TV shows
rating_list = []
for i in range(len(containers_rating)):
    rating = containers_rating[i].strong["title"]
    rating = rating[:3]
    rating_list.append(rating)
    
#listing out the encoded title of all the TV shows
tv_list = []
for i in range(0, len(containers)):
    title = containers[i].a['href']
    title = title.split("/")[2]
    tv_list.append(title)

In [57]:
#creating a list that extracts information about each TV show such as
#title, rating, total_votes, description, release year, its link and its encoded title
comprehensive_list = []
for tv in tv_list:
    response = get('https://www.imdb.com/title' + tv + '/')
    tv_soup = BeautifulSoup(response.text, 'html.parser')
    title_verbose = tv_soup.find('title').string
    releaseYear = re.findall(r'[0-9][0-9][0-9][0-9]', title_verbose)
    tv_soup = BeautifulSoup(response.text, 'html.parser')
    #tv_title = tv_soup.find('title').string
    #rating = tv_soup.find("span", class_ = "AggregateRatingButton_RatingButton_RatingScore-sc-111290-1 iTLWoV").string
    rating_count = tv_soup.find("div", class_="sc-7ab21ed2-3 dPVcnq")
    tv_title = tv_soup.find("h1", {"data-testid": "hero-title-block__title"})
    description = tv_soup.find("span", {'data-testid': "plot-xl"})
    link = 'https://www.imdb.com/title' + tv
    encoded_title = tv
    genres = tv_soup.find_all('a', class_ = "GenresAndPlot_GenreChip-sc-cum89p-3 LKJMs ipc-chip ipc-chip--on-baseAlt")
    genre = ""
    for g in genres:
        genre += g.text + ", "
    genre = genre.strip(", ")
    comprehensive_list.append([tv_title, rating_count, description, releaseYear[], link, encoded_title, genre])


IndexError: list index out of range

In [54]:
#converting comprehensive_list into a data frame
tv_best = pd.DataFrame(comprehensive_list, columns = ["title", "total_votes", "show_desc", "year", "link", "encoded_title", "genre"])

#saving the rating list to a column
tv_best["rating"] = rating_list
tv_best["rating"] = tv_best["rating"].astype(float)
tv_best = tv_best[["title", "rating", "total_votes", "show_desc", "year", "link", "genre", "encoded_title"]]

In [55]:
#converting K to 000 for total votes and M to 000000 for total votes
tv_best["total_votes"] = tv_best["total_votes"].str.replace("K", "000")
tv_best["total_votes"] = tv_best["total_votes"].str.replace("M", "000000")

tv_best.head()

,title,rating,total_votes,show_desc,year,link,genre,encoded_title
0,None,9.4,None,None,[],https://www.imdb.com/titlett5491994,,tt5491994
1,None,9.4,None,None,[],https://www.imdb.com/titlett0903747,,tt0903747
2,None,9.4,None,None,[],https://www.imdb.com/titlett0795176,,tt0795176
3,None,9.4,None,None,[],https://www.imdb.com/titlett0185906,,tt0185906
4,None,9.3,None,None,[],https://www.imdb.com/titlett7366338,,tt7366338
